In [3]:
import torch
import torchvision
import torch.nn as nn
from torchvision.models import VGG16_Weights

# 测试数据集读取
## 自定义数据集

In [4]:
from loadData import load_data
train_loader, test_loader, database_loader, train_num, test_num, database_num= load_data(250,4)

ImportError: attempted relative import with no known parent package

In [11]:
k = 0
for img ,cls, i in train_loader:
  if k == 3 :
    break
  print("索引:",i)
  print("图像:",img)
  print("类别",cls)
  print("索引shape:",i.shape)
  print("图像shape:",img.shape)
  print("类别shape:",cls.shape)
  k = k+1

索引: tensor([45551, 47879, 27007,  6802,  7026,  9705, 17951, 24817,  4233, 35583,
        25490, 32287, 25501, 36654, 19097, 38050, 12021, 12316, 49738, 13537,
        47244, 25444, 36763, 30778, 14147, 20284, 17486,  7989, 37191, 44754,
        49472, 19633, 13542, 33630,  4966, 29463,  4090, 30270, 33944, 30564,
        14836, 16287, 27826,  8560, 43842, 10528, 39054, 29526,  4664, 12761,
        48630, 30731,  6330, 48900, 35152, 11001,   993, 30273, 26595, 11979,
        33635, 15795, 41101,  6197, 29386, 44940, 29763, 25040, 33155, 38413,
        27314, 24102, 16632, 18295, 24160, 43678, 25223, 38922, 38183, 32320,
         9999, 19468, 47069, 43143,   580,  6933, 26184, 35301, 47014, 44472,
        22948,  1028, 48393, 42772, 16800, 21328, 43995, 26295, 30074, 44662,
         9592, 20355, 45864, 34638, 17160, 32541, 39219, 12033,  9917, 28446,
        25282, 12355, 21096,  1803, 47530, 30993, 31766, 20074, 34228, 45688,
        35677, 24216, 39140,  5799,  3838, 21856, 35149, 113

## pytorch内置数据集

In [16]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    # transforms.Resize(args.crop_size),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
train_ds = datasets.CIFAR10(
  root="dataset/cifar",
  train=True,
  download=False,
  transform=transform,
)
test_ds = datasets.CIFAR10(
  root="dataset/cifar",
  train=False,
  download=False,
  transform=transform,
  target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
)

In [17]:
train_dataloader = DataLoader(train_ds, batch_size=250, shuffle=True)
test_dataloader = DataLoader(test_ds, batch_size=250, shuffle=False)

In [18]:
k=0
for img ,cls, i in train_loader:
  if k == 3 :
    break
  print("索引:",i)
  print("图像:",img)
  print("类别",cls)
  print("索引shape:",i.shape)
  print("图像shape:",img.shape)
  print("类别shape:",cls.shape)
  k = k+1

索引: tensor([38704,  4208, 36667, 48153, 13591, 43891,  8730,  1204, 44386,  3254,
        36458, 45825, 14427, 28404,  7381, 42207, 33917, 44880, 36606, 15333,
        44628, 32877, 37943, 22500, 42101, 47862, 37718, 30597,  3264, 46677,
        17466, 30600,   619, 10863, 16665, 33925, 44165, 21366, 46032, 35883,
        34302,  2119, 18002, 21350, 36311, 17449, 36888, 26587,  7192, 15958,
        33305, 12115, 39954, 15936, 23145, 34914,  6086, 48018,  3539, 30750,
         8759, 47864, 48312, 30719, 36172, 16761, 38205, 18299, 45769,  6231,
        47394, 19833, 17132, 34320, 15472, 24483,  8878, 27365, 49198, 15582,
        17113,  2926, 18311,  4104, 13388, 36650,  8635, 21633, 46927, 24103,
        39544, 23183, 36712, 37506,  5387, 34404, 29499, 34970, 19997, 33954,
        13375,  4664, 40402,  6809, 49210, 43003, 34443, 49451,  4749,  8311,
        33684, 16296, 24755, 36877, 30739, 30643, 34336, 16943, 26447, 25368,
         4222, 30448, 35034, 47052,  4527,  5067, 13565, 392

In [3]:
from nus_wide import load_nuswide_data
load_nuswide_data(21, 5000, 1000, 256, 4)

FileNotFoundError: [Errno 2] No such file or directory: '/data2/fyang/dataset/nus-wide/test_img.txt'

# NUSWIDE数据集划分

In [1]:
import numpy as np
import scipy.io as sio
import os
from os.path import join
import time
import matplotlib.pyplot as plt

In [2]:
NUSWIDE = "/data2/fyang/dataset/NUS-WIDE"
labels = sio.loadmat(join(NUSWIDE, "nus-wide-tc21-lall.mat"))["LAll"]
print(labels.shape)
N_CLASS = labels.shape[1]
N_SAMPLE = labels.shape[0]
TEST_PER = 100  # test set 每个类 100 个
TRAIN_PER = 500  # training set 每个类 500 个
N_TEST = TEST_PER * N_CLASS
N_TRAIN = TRAIN_PER * N_CLASS


"""1. 先保证 test set 的每类至少 100"""
indices = list(range(N_SAMPLE))  # 全部索引
np.random.shuffle(indices)

cls_sum = np.sum(labels[indices], axis=0)  # 统计每个类样本数
#print(cls_sum)
classes = np.argsort(cls_sum)  # 从少到多
#print(classes)

id_test = []
cnt = np.zeros_like(labels[0], dtype=np.int32)  # 默认 int8，会爆
for cls in classes:
    print("--- {} ---".format(cls))
    for i in indices:
        if cnt[cls] >= TEST_PER:  # 此类已抽够
            break
        if labels[i][cls] == 1:
            id_test.append(i)
            cnt += labels[i]
    #print(cnt)
    assert cnt[cls] >= TEST_PER  # 讲道理一趟下来是肯定够的
    indices = list(set(indices) - set(id_test))  # 去掉已抽部分的 id
    np.random.shuffle(indices)
    #print("left:", len(indices))

assert len(set(id_test)) == len(id_test)  # 验证没有重复
#print("cnt:", cnt)
print("#test:", len(id_test))


"""2. 类似地，保证 training set 的每类至少 500"""
indices = list(set(indices) - set(id_test))  # 去掉刚才选过的那些 test id
np.random.shuffle(indices)
print(len(indices))

cls_sum = np.sum(labels[indices], axis=0)
#print(cls_sum)
classes = np.argsort(cls_sum)
#print(classes)

id_train = []
cnt = np.zeros_like(labels[0], dtype=np.int32)
for cls in classes:
    print("--- {} ---".format(cls))
    for i in indices:
        if cnt[cls] >= TRAIN_PER:
            break
        if labels[i][cls] == 1:
            id_train.append(i)
            cnt += labels[i]
    #print(cnt)
    assert cnt[cls] >= TRAIN_PER
    indices = list(set(indices) - set(id_train))
    np.random.shuffle(indices)
    #print("left:", len(indices))

assert len(set(id_train)) == len(id_train)
#print("cnt:", cnt)
print("#train:", len(id_train))


"""3. 补足 test 和 training set 剩余的部分"""
indices = list(set(indices) - set(id_train))  # 再去掉刚才选过的 train id
np.random.shuffle(indices)
#print(len(indices))

lack_test = N_TEST - len(id_test)
lack_train = N_TRAIN - len(id_train)
print("lack:", lack_test, ",", lack_train)

id_test.extend(indices[:lack_test])
id_train.extend(indices[lack_test: lack_test + lack_train])

print("#total test:", len(id_test))
print("#total train:", len(id_train))


"""4. unlabeled 部分"""
# unlabeled = all - labeled(training) - query(test)
id_unlabeled = list(set(indices) - set(id_train) - set(id_test))
print("#unlabeled:", len(id_unlabeled))


"""5. retrieval set"""
id_ret = id_train + id_unlabeled
print("#retrieval:", len(id_ret))


"""保存"""
_info = "nuswide-tc21.{}pc.{}pc".format(TEST_PER, TRAIN_PER)
SAV_P = join(NUSWIDE, _info)
if not os.path.exists(SAV_P):
    os.makedirs(SAV_P)

test_id = np.asarray(id_test)
labeled_id = np.asarray(id_train)
unlabeled_id = np.asarray(id_unlabeled)
ret_id = np.asarray(id_ret)

np.save(join(SAV_P, "idx_test.npy"), test_id)
np.save(join(SAV_P, "idx_labeled.npy"), labeled_id)
np.save(join(SAV_P, "idx_unlabeled.npy"), unlabeled_id)
np.save(join(SAV_P, "idx_ret.npy"), ret_id)

(195834, 21)
--- 20 ---
--- 19 ---
--- 18 ---
--- 17 ---
--- 16 ---
--- 15 ---
--- 14 ---
--- 13 ---
--- 12 ---
--- 11 ---
--- 10 ---
--- 9 ---
--- 8 ---
--- 7 ---
--- 6 ---
--- 5 ---
--- 4 ---
--- 3 ---
--- 2 ---
--- 1 ---
--- 0 ---
#test: 994
194840
--- 20 ---
--- 19 ---
--- 18 ---
--- 17 ---
--- 16 ---
--- 15 ---
--- 14 ---
--- 13 ---
--- 12 ---
--- 11 ---
--- 10 ---
--- 9 ---
--- 8 ---
--- 7 ---
--- 6 ---
--- 5 ---
--- 4 ---
--- 3 ---
--- 2 ---
--- 1 ---
--- 0 ---
#train: 4733
lack: 1106 , 5767
#total test: 2100
#total train: 10500
#unlabeled: 183234
#retrieval: 193734


In [7]:
mat_data = np.load('/data2/fyang/dataset/NUS-WIDE/nus-wide-tc21-IAll/IAll.npy')
loadData = np.load('/data2/fyang/dataset/NUS-WIDE/nuswide-tc21.100pc.500pc/idx_test.npy')
mat_data

array([[[[254, 254, 254],
         [251, 254, 248],
         [250, 254, 247],
         ...,
         [157, 148, 156],
         [209, 204, 206],
         [255, 255, 255]],

        [[255, 249, 248],
         [237, 236, 231],
         [228, 229, 227],
         ...,
         [ 45,  59,  46],
         [136, 145, 133],
         [249, 249, 247]],

        [[255, 251, 252],
         [188, 189, 189],
         [152, 156, 160],
         ...,
         [ 63,  65,  63],
         [127, 133, 132],
         [221, 221, 219]],

        ...,

        [[254, 250, 252],
         [235, 234, 236],
         [230, 233, 233],
         ...,
         [156, 168, 166],
         [199, 204, 205],
         [234, 234, 233]],

        [[254, 254, 248],
         [249, 253, 247],
         [247, 252, 246],
         ...,
         [244, 235, 238],
         [249, 242, 244],
         [251, 251, 251]],

        [[255, 251, 254],
         [253, 253, 254],
         [252, 254, 254],
         ...,
         [223, 227, 227],
        

In [74]:
idx_test = np.load('/data2/fyang/dataset/NUS-WIDE/nuswide-tc21.100pc.500pc/idx_test.npy')
idx_train = np.load('/data2/fyang/dataset/NUS-WIDE/nuswide-tc21.100pc.500pc/idx_labeled.npy')
idx_retrieval = np.load('/data2/fyang/dataset/NUS-WIDE/nuswide-tc21.100pc.500pc/idx_ret.npy')

In [75]:
print(f"test size: {idx_test.shape}")
print(f"train size: {idx_train.shape}")
print(f"retrieval size: {idx_retrieval.shape}")

test size: (2100,)
train size: (10500,)
retrieval size: (193734,)


In [76]:
import torch
test_data = mat_data[idx_test]
train_data = mat_data[idx_train]
retrieval_data = mat_data[idx_retrieval]

In [59]:
np.save("/data2/fyang/dataset/NUS-WIDE/nuswide-tc21.100pc.500pc/img_test.npy", test_data.transpose(0, 3, 1, 2))
np.save("/data2/fyang/dataset/NUS-WIDE/nuswide-tc21.100pc.500pc/img_train.npy", train_data.transpose(0, 3, 1, 2))
np.save("/data2/fyang/dataset/NUS-WIDE/nuswide-tc21.100pc.500pc/img_retrieval.npy", retrieval_data.transpose(0, 3, 1, 2))


In [69]:
temp = np.load("/data2/fyang/dataset/NUS-WIDE/nuswide-tc21.100pc.500pc/img_test.npy")
temp.shape

(2100, 3, 224, 224)

In [19]:
import scipy.io as sio
labels = sio.loadmat('/data2/fyang/dataset/NUS-WIDE/index.mat')
labels

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Wed Dec 27 15:01:39 2023',
 '__version__': '1.0',
 '__globals__': [],
 'indQ': array([[ 97403,  29571,  53852, ..., 158902,  79069,  99998]]),
 'indR': array([[ 41059,  60341,  52845, ..., 121909, 165158, 189139]]),
 'indT': array([[ 41059,  60341,  52845, ..., 106793, 110620,  80628]])}

In [29]:
label_all = sio.loadmat('/data2/fyang/dataset/NUS-WIDE/nus-wide-tc21-lall.mat')
label_all = label_all['LAll']
label_all.shape

(195834, 21)

In [53]:
test_label = label_all[loadData]
test_label

array([[1, 0, 0, ..., 0, 0, 1],
       [1, 1, 1, ..., 0, 0, 1],
       [1, 0, 0, ..., 1, 0, 1],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [50]:
from PIL import Image
tensor1[3].shape
tensor_image = tensor1[3].to(torch.uint8).numpy()
image = Image.fromarray(tensor_image)
image.save('output_image.png')

In [1]:
from nus_wide import load_nuswide_data
train_loader, test_loader, dataset_loader = load_nuswide_data(21, 10500, 2100, 128,4)

train_dataset 10500
query_dataset 2100
retrieval_dataset 193734
